In [31]:
import pandas as pd
import numpy as np


In [6]:
from gameplan.growth.data_sources import KitcesData


In [ ]:
from gameplan.user import User
from gameplan.growth.growth_models import KitcesIncomeGrowthModel
from gameplan.income_streams import Salary

In [86]:
def get_percentile_label(x):
    pctile = round(x) # sp.percentile returns percentiles out of 100, not 1
    last_digit = str(pctile)[-1]
    suffix_dict = {
        '1' : 'st',
        '2' : 'nd',
        '3' : 'rd'
    }

    return f"{pctile}{suffix_dict.get(last_digit, 'th')}"

def construct_subset_query(dma, age_range, gender):
    if gender == 'All':
        gender_clause = "(sex == 'Male' | sex == 'Female')"
    else:
        gender_clause = f"sex == '{gender}'"
    query = f" \
        metarea.str.contains(@dma) \
        & AGE.between(@age_range[0], @age_range[1]) \
        & {gender_clause} \
        "
    return query

def get_cohort_subset(df, dma, age_range, gender):
    query = construct_subset_query(dma, age_range, gender)
    return df.query(query)

def get_pctile_for_growth(x):
    "Current growth model only has 0, 10 ... 90th percentiles"
    if x == 100:
        x -= 1
    return np.floor(x/10)*10

def get_salary_trajectory(pctile_for_growth, age, current_salary):
    user = User(
        email='placeholder',
        birthday=pd.datetime.today() - pd.DateOffset(years=age),
        income_percentile=pctile_for_growth
        )

    sal_grwth_points = user.get_growth_points_to_fit(
        growth_model=KitcesIncomeGrowthModel,
        start_dt=pd.datetime.today()
    )
    s = Salary(
        current_salary,
        payday_freq='Y',
        growth_points_to_fit=sal_grwth_points,
        last_paycheck_dt= pd.datetime.today() + pd.DateOffset(years=65 - age),
        tax_rate=.35
    )
    return s

def get_growth_scenarios(percentile, age, current_salary):
    pctile_for_growth = get_pctile_for_growth(percentile)
    growth_scenarios = {
        'Optimistic': {'grwth_pctile': pctile_for_growth + 10},
        'Status Quo': {'grwth_pctile': pctile_for_growth},
        'Pessimistic': {'grwth_pctile': pctile_for_growth - 10},
    }
    # We don't have an "optimistic" scenario if you're in the 90th pctile
    if pctile_for_growth >= 90:
        growth_scenarios.pop('Optimistic')
    elif pctile_for_growth <= 10:
        growth_scenarios.pop('Pessimistic')


    for scn, v in growth_scenarios.items():
        v['sal_traj'] = get_salary_trajectory(v['grwth_pctile'], age, current_salary)

    return growth_scenarios


def get_income_trajectory_fig(df, salary, dma, age_range, gender, age):
    subset = get_cohort_subset(df, dma, age_range, gender)
    dist = get_cdf(subset)
    percentile = sp.percentileofscore(dist.inctot, int(salary))
    growth_scenarios = get_growth_scenarios(percentile, age, salary)


    return px.scatter(s.paycheck_df.salary.reset_index(), 'index', 'salary')


In [165]:
blah = get_growth_scenarios(80, 29, 118000)

In [166]:
s = blah['Optimistic']['sal_traj']

In [167]:
for scn in blah:
    s = blah[scn]['sal_traj']
#     px.scatter(s.paycheck_df.salary.reset_index(), 'index', 'salary')

In [168]:
import plotly.graph_objects as go

In [170]:
birthday = pd.datetime.today() - pd.DateOffset(years=29)
blah['Status Quo']['sal_traj'].paycheck_df.index.to_series().apply(lambda x: (x - birthday)).iloc[0]

/Users/davidbrodsky/repos/gameplan/gameplan/gp_collections.py:114: UserWarning:

This Collection is empty.



Timedelta('10595 days 07:22:59.028265')

In [171]:
age_index = [round((dt-birthday).days/365) for dt in s.date_range]

In [203]:
data = [
    go.Scatter(
        x=v['sal_traj'].paycheck_df.index,
        y=v['sal_traj'].paycheck_df.salary,
        name=f"{scn} Growth Trajectory - {int(v['grwth_pctile'])}th percentile",
        mode='markers+lines',
        xaxis='x'
    )
    for scn, v in blah.items()
]

data.append(
    go.Scatter(x=age_index, y=[0]*len(age_index), xaxis='x2', visible='legendonly', 
               showlegend=False)
)

layout = go.Layout(
    title=go.layout.Title(text=''),
    xaxis=go.layout.XAxis(
        range=[min(s.date_range) - pd.DateOffset(years=1), max(s.date_range) + pd.DateOffset(years=1)],
        
    ),
    xaxis2=go.layout.XAxis(
        title='Age',
        overlaying='x', 
        side='top',
        dtick=2,
#         tickvals=[x for x in age_index if x%5 == 0 ],
        range=[min(age_index) - 1, max(age_index) + 1]
    ),
    yaxis=go.layout.YAxis(
        title='Pre-Tax Income in 2019 Dollars (inflation-adjusted)'
    ),
    legend=go.layout.Legend(orientation='h', y=-0.3),   
)


go.Figure(data, layout)

/Users/davidbrodsky/repos/gameplan/gameplan/gp_collections.py:114: UserWarning:

This Collection is empty.



In [94]:
pd.concat([s.paycheck_df

/Users/davidbrodsky/repos/gameplan/gameplan/gp_collections.py:114: UserWarning:

This Collection is empty.



,salary,total_deductions,total_taxes,take_home_salary
2019-12-31,118000.000000,0.0,-41300.000000,76700.000000
2020-12-31,124162.276013,0.0,-43456.796604,80705.479408
2021-12-31,130289.144405,0.0,-45601.200542,84687.943863
2022-12-31,136368.469582,0.0,-47728.964354,88639.505228
2023-12-31,142371.280183,0.0,-49829.948064,92541.332119
2024-12-31,148284.590438,0.0,-51899.606653,96384.983784
2025-12-31,154047.049632,0.0,-53916.467371,100130.582261
2026-12-31,159646.000795,0.0,-55876.100278,103769.900517
2027-12-31,165052.472564,0.0,-57768.365397,107284.107166
2028-12-31,170251.368360,0.0,-59587.978926,110663.389434


In [51]:
def get_pctile_for_growth(x):
    if x == 100:  # there's no 
        x -= 1
    return np.floor(x/10)*10

In [53]:
user = User(
    email='placeholder',
    birthday=pd.datetime.today() - pd.DateOffset(years=29),
    income_percentile=get_pctile_for_growth(90)
    )

sal_grwth_points = user.get_growth_points_to_fit(
    growth_model=KitcesIncomeGrowthModel,
    start_dt=pd.datetime.today()
)
s = Salary(
    salary, 
    payday_freq='Y', 
    growth_points_to_fit=sal_grwth_points, 
    last_paycheck_dt= pd.datetime.today() + pd.DateOffset(years=65 - age)
    tax_rate=.35
)

In [65]:
pd.datetime.today() + pd.DateOffset(years=65 - 29)

Timestamp('2055-12-27 12:16:58.785661')

In [60]:
import plotly.express as px


In [69]:
s.paycheck_df

/Users/davidbrodsky/repos/gameplan/gameplan/gp_collections.py:114: UserWarning:

This Collection is empty.



,salary,total_deductions,total_taxes,take_home_salary
2019-12-31,180000.000000,0.0,-63000.000000,117000.000000
2020-12-31,193886.573605,0.0,-67860.300762,126026.272843
2021-12-31,207931.691236,0.0,-72776.091932,135155.599303
2022-12-31,222101.400730,0.0,-77735.490255,144365.910474
2023-12-31,236324.142608,0.0,-82713.449913,153610.692695
2024-12-31,250567.181978,0.0,-87698.513692,162868.668286
2025-12-31,264680.965077,0.0,-92638.337777,172042.627300
2026-12-31,278632.906069,0.0,-97521.517124,181111.388945
2027-12-31,292351.445473,0.0,-102323.005916,190028.439557
2028-12-31,305801.288820,0.0,-107030.451087,198770.837733


In [72]:
fig = px.scatter(s.take_home_salary.reset_index(), 'index', 'take_home_salary')
# fig.add_scatter(s.take_home_salary.reset_index(), 'index', 'take_home_salary')
fig

In [28]:
me = User(
    'placeholder', 
    pd.datetime.today() - pd.DateOffset(years=29), 
    income_percentile=80
)

In [45]:
get_pctile(100)

90.0

In [29]:
me.get_growth_points_to_fit(KitcesIncomeGrowthModel)

[(0, 1.0),
 (366, 1.0584595986007068),
 (2192, 1.3807438602032824),
 (4018, 1.594454051419798),
 (5844, 1.828753099515052),
 (7671, 1.9459435367778166),
 (9497, 1.823626918786438),
 (11323, 1.6492554967794082)]

In [14]:
kigm = KitcesIncomeGrowthModel(me.birthday, income_percentile=me.income_percentile)
# me.get_growth_points_to_fit()

In [19]:
me.get_growth_points_to_fit(KitcesIncomeGrowthModel)

[(0, 1.0),
 (258, 1.0669881824391954),
 (2084, 1.4680536272267197),
 (3910, 1.7724145959272346),
 (5736, 2.1244655358330493),
 (7563, 2.3090183155729505),
 (9389, 2.254131326381477),
 (11215, 2.0815939556498404)]

In [17]:
kigm.get_growth_points_to_fit(start_dt=pd.datetime(2020, 11, 1))

[(0, 1.0),
 (1774, 1.3756874512010506),
 (3600, 1.6608988069113084),
 (5426, 1.9907996029243749),
 (7253, 2.1637407942159137),
 (9079, 2.112307153874343),
 (10905, 1.9506253928155697)]

In [ ]:
go.layout.XAxis()